In [1]:
import jax
import jax.numpy as jnp

In [4]:
rng = jax.random.PRNGKey(0)

# (R, dk)
K = jax.random.uniform(rng, (6, 4))

# query: (B, d_k)
q = jnp.ones((2, 4))

# values: (num_values, d_v)
v = jnp.ones((6, 5))

# (num_values, num_keys)
neighborhood = jnp.array([
    [0, 0, 1, 1, 1, 0],
    [1, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1]
])


### Doing atn the dumb way

In [8]:
def atn_naive(q, k, v, neighborhood):
    R = k.shape[0]
    B, dk = q.shape
    print(f"B: {B}, R: {R}, V: 4, dk: {dk}")
    QK = jnp.exp((k @ q.T).T / dk)

    print("QK: ", QK.shape)
    
    # (batch, num_values, num)
    QK_masked = jnp.expand_dims(QK, 1) * neighborhood
    print(QK_masked.shape)
    print(QK_masked)
    norm_terms = jnp.sum(QK_masked, axis=2)
    # print(norm_terms)
    scores = QK_masked / jnp.expand_dims(norm_terms, 2)
    v = jnp.expand_dims(v, 0) 

    # (batch, num_keys, num_values)
    scores = jnp.transpose(scores, (0, 2, 1))
    # print(scores.shape, v.shape)
    res = scores @ v
    # print(res.shape)
    return res

atn_naive(q, K, v, neighborhood)

B: 2, R: 6, V: 4, dk: 4
QK:  (2, 6)
(2, 4, 6)
[[[0.        0.        1.5542811 1.624569  1.6013266 0.       ]
  [1.4231399 1.2928791 0.        0.        0.        0.       ]
  [0.        0.        0.        0.        0.        0.       ]
  [0.        0.        0.        0.        0.        2.1730514]]

 [[0.        0.        1.5542811 1.624569  1.6013266 0.       ]
  [1.4231399 1.2928791 0.        0.        0.        0.       ]
  [0.        0.        0.        0.        0.        0.       ]
  [0.        0.        0.        0.        0.        2.1730514]]]


TypeError: dot_general requires contracting dimensions to have the same shape, got (4,) and (6,).